In [1]:
using POMDPs, POMDPTools, QuickPOMDPs, MCTS, DiscreteValueIteration, POMDPSimulators, POMDPModels, POMDPPolicies
using Distributions, Combinatorics, StaticArrays, D3Trees, Random
using FileIO, JLD2, TickTock

In [2]:
action=[(0,0), (0,1), (1,1)]

global Number_level = 4;
global Transition_matrix = [
    0.8571 0.1429 0.0    0.0;
    0.0    0.8571 0.1429 0.0;
    0.0    0.0    0.8    0.2;
    0.0    0.0    0.0    1.0
]
failure_penalty = -1000.0;
maintenance_penalty = -200.0;
setup_cost = -800.0;
system_penalty = - 2000.0;
normal_operation = 0.0;

In [3]:
# Note that the reward function here is only decided by component-wise independent knowledge, 
# i.e., we know nothing about other components when making decision for this component
function reward(s, a,s1=Number_level, rng=23)       #s is a vector of number units at each level and a is the number of units we will repair
    #    if s==Number_level
    #         #r=failure_penalty  + normal_operation  +  setup_cost/NumberUnits  + system_penalty/NumberUnits;
    #         r=system_penalty/limit
    #     return r
    #    end
    #     r=0
     if a==1 # don't replace component
        if Number_level>s
            crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r = normal_operation ;
         return r
        else
            r=system_penalty/NumberUnits;
            return r
        end
        end

    if a==2 # replace component
        if Number_level>s
            s=1
            crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r = maintenance_penalty+  setup_cost/NumberUnits;
         return r
        else
            s=1
            crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r= failure_penalty  +  setup_cost/NumberUnits;
        return r
        end
        end
    end


reward (generic function with 3 methods)

In [4]:
s=1:Number_level;

In [5]:
T=zeros(Number_level,Number_level,3);


In [6]:
T[:,:,1].=Transition_matrix;

In [7]:
T[:,:,2].=Transition_matrix;

In [8]:
for j in 1:Number_level
        T[j,:,2].=Transition_matrix[1,:];
        end
# for j in 1:10
#         T[j,:,3].=0.0
#     T[j,1,3]=1.0
#         end

In [9]:
T[Number_level,:,1].=Transition_matrix[1,:]
T[Number_level,:,2].=Transition_matrix[1,:]

4-element view(::Array{Float64, 3}, 4, :, 2) with eltype Float64:
 0.8571
 0.1429
 0.0
 0.0

In [10]:
function explore()
    c=[1/3,1/3,1/3]

     crd = Categorical(c);
     return rand(crd)
    end 
function generative(s, a, rng=23)       #s is a vector of number units at each level and a is the number of units we will repair

r=0
    if a==1 # don't replace component
        if Number_level>s
            crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r = normal_operation ;
            return (sp=s, r=r)
        else
            r=0;
            return (sp=s, r=r)
        end
        end

    if a==2 # replace component
        if Number_level==s
            s=1
            crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r= failure_penalty;
            
            return (sp=s, r=r)
        end
        if s==1
            crd = Categorical(Transition_matrix[s,:]);
            s = rand(crd);
            r= 0;
            return (sp=s, r=r)
        end
        s=1
        crd = Categorical(Transition_matrix[s,:]);
        s = rand(crd);
        r = maintenance_penalty;
        return (sp=s, r=r)
    end
    end

generative (generic function with 2 methods)

In [11]:
# the value iteration here is simply based on independent component-level knowledge
function ValueIteration(S, T, γ, p1=0.01, d=100)
    Q=zeros((Number_level,3))
#     print(Q)
    V= rand(Float64, (Number_level))*1000
    diff=0
    i=0
    
    

    
    p=[1 1 1 ]
    while i<=2000
#     while  i<=10000
    for s in 1:Number_level
        Threads.@threads for a in 1:2
 
                Q[s,a]= (reward(s,a)) + γ*( sum(T[s,s1,a]*(V[s1]) for (s1) in 1:Number_level ))
    end
            V[s]=findmax(Q[s,1:2])[1]
        end
        i+=1
        end
    return Q,V
    end

ValueIteration (generic function with 3 methods)

In [12]:


function decison(Q, s,n=0.2)
   a_final=[]
   if true
        for i in 1:NumberUnits
            append!(a_final,argmax(a->Q[s[i],a], [1,2]))
            end
        return a_final
    end
end

decison (generic function with 2 methods)

In [13]:

using DataFrames
df= DataFrame(u=[],K=[],n=[],s=[],m=[],f=[],p=[],lamb=[],mean=[],std=[])
cost=[
    [0	-600	-100	-1200 -2000],
    [0	-800	-100	-1200 -2000],
    [0	-1000	-100	-1200 -2000],
    [0	-1400	-100	-1200 -2000],
    [0	-1800	-100	-1200 -2000],
    [0  -2400   -100    -1200 -2000]
] 

for units in [150,140,130,120,110,100,90,80,70,60,50,40,30,20]
    for K in 2:convert(Int64, floor((units+1)/10)):convert(Int64, floor((units+1)/2))
        global NumberUnits=convert(Int64,units)
        global limit=convert(Int64, K)
    for c in cost
   print(units)  
   print("K : ")
    println(K)
    global normal_operation,setup_cost,maintenance_penalty,failure_penalty,system_penalty=c
     println([normal_operation,setup_cost,maintenance_penalty,failure_penalty,system_penalty])   
     for lambda in 1:1:1
        global  lamb=lambda
     Q,V=ValueIteration(s,T,.95,3000)
         println(isnan(Q[1,1]))
       if sum(isnan(Q[1,1]))>0.0
            continue
        end
tick()
trials=10000;
simsteps = 100;
results=zeros(trials)*0.1
Threads.@threads for h in 1:trials
s=repeat(1:1,NumberUnits)
r1=[]
a1=[]   
r=repeat(1:1,NumberUnits)*1.0
 for k in 1:simsteps
    a =decison(Q,s)
    r2=0.0
    s1=repeat(1:1,NumberUnits)
              
# Here we are calculating the "true" system-level rewards, following formulation (3) in original paper
# i.e., adding up reward of each component, and decide to add setup cost or not depending on system-level condition 
# In contrast, the policy we are using is obtained only at component level          
#     print(a)
global failed_components = 0
    for i in 1:NumberUnits
        if s[i] == Number_level
            failed_components += 1
         end
#             print(generative(s[i],a[i],234))
         s1[i],r[i]=generative(s[i],a[i],234);
          r2+=r[i]
        
            end 
#     println(r2)
#     println(s)

    if  (2 in a) 
        # if any replacement
            r2+=setup_cost
                    end
    if failed_components >= limit
        r2+=system_penalty
    end
    s=s1;
    append!(a1,a)
    append!(r1,r2)
end
rk=0
for i in 1:simsteps
    rk=0.95*rk+r1[simsteps-i+1]
end
#     println(rk)
    results[h]=rk
#     println(results)
end
println(units)
println(lambda)

println(mean(results))

println(std(results))
pushfirst!(df,[units,limit,normal_operation,setup_cost,maintenance_penalty,failure_penalty,system_penalty,lamb,mean(results),std(results)])
        
tock()
end
    println("-------------------------")
end
end
end
end

150K : 2
[0, -600, -100, -1200, -2000]
false


[ Info:  started timer at: 2024-03-27T18:58:35.437


150
1
-39161.28645860959
806.151256993156
-------------------------
150K : 2
[0, -800, -100, -1200, -2000]


[ Info:           41.8089399s: 41 seconds, 808 milliseconds


false


...

false


[ Info:  started timer at: 2024-03-27T21:43:46.758


20
1
-21344.936336418712
3406.596018559216
-------------------------


[ Info:            4.7926072s: 4 seconds, 792 milliseconds


LoadError: syntax: unexpected "end"

In [14]:
df

Row,u,K,n,s,m,f,p,lamb,mean,std
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,20.0,10.0,0.0,-2400.0,-100.0,-1200.0,-2000.0,1.0,-21344.9,3406.6
2,20.0,10.0,0.0,-1800.0,-100.0,-1200.0,-2000.0,1.0,-21433.5,3449.1
3,20.0,10.0,0.0,-1400.0,-100.0,-1200.0,-2000.0,1.0,-21347.4,3461.58
4,20.0,10.0,0.0,-1000.0,-100.0,-1200.0,-2000.0,1.0,-21390.0,3434.8
5,20.0,10.0,0.0,-800.0,-100.0,-1200.0,-2000.0,1.0,-21324.3,3465.46
6,20.0,10.0,0.0,-600.0,-100.0,-1200.0,-2000.0,1.0,-21232.0,3388.32
7,20.0,8.0,0.0,-2400.0,-100.0,-1200.0,-2000.0,1.0,-24295.3,3368.47
8,20.0,8.0,0.0,-1800.0,-100.0,-1200.0,-2000.0,1.0,-24255.4,3356.92
9,20.0,8.0,0.0,-1400.0,-100.0,-1200.0,-2000.0,1.0,-24200.4,3366.76


In [15]:
using CSV
CSV.write("Independent solution(case study).csv", df)

"Independent solution(case study).csv"

In [16]:
#=  Q=ValueIteration(s,T,.95,3000)
for s in 1:Number_level
p=[exp(0.01*sum(Q[s,1])),exp(0.01*sum(Q[s,2])),exp(0.01*sum(Q[s,3]))]
    p./=sum(p)
    println(p)
end =#

In [17]:
#= using Plots =#

In [18]:
#= for i in [3,5,10,15,20,25,30,40,50,60]
   display( plot(df[df.Units.==i,:].lambda,df[df.Units.==i,:].mean, xtick=df[df.Units.==i,:].lambda, show = true,title=string(i),label=false, xaxis=:log10,rotation=40))
    plot(show=true)
    println(df[df.Units.==i,:][findmax(df[df.Units.==i,:].mean)[2],:])

end =#

In [19]:
#= plot!()
 =#